In [7]:
import numpy as np
import math
import sys

In [9]:
def F(x,y,z):
  return (x & y) | (~x & z)

def G(x,y,z):
  return (x & z) | (y & ~z)

def H(x,y,z):
  return x ^ y ^ z

def I(x,y,z):
  return y ^ (x | ~z)

def left_rotate(x, amount):
    x &= 0xFFFFFFFF
    return (((x<<amount))| (x>>(32-amount))) & 0xFFFFFFFF

def md5_to_hex(digest):
    raw = digest.to_bytes(16, byteorder='little')
    return '{:032x}'.format(int.from_bytes(raw, byteorder='big'))

def custom_md5(m, h0):
  T = [ 0 for i in range(64)]
  m = str.encode(m)
  a0 = (h0 // (2 ** (32 * 3))) % (2 ** 32)
  b0 = (h0 // (2 ** (32 * 2))) % (2 ** 32)
  c0 = (h0 // (2 ** 32)) % (2 ** 32)
  d0 = h0 % (2 ** 32)

  constantes = [a0,b0 ,c0 ,d0]
  message = bytearray(m)
  leninbits = (8 * len(message)) & 0xffffffffffffffff
  message.append(0x80)
  while len(message)% 64 != 56:
    message.append(0)
  message += leninbits.to_bytes(8, byteorder="little")


  for i in range(0, 64):
    T[i] = int(abs(math.sin(i+1)) * 2**32) & 0xFFFFFFFF 

  a,b,c,d = constantes[:]
  for p in range(0, len(message) ,64):
    x = message[p:p+64]
    words = []
    for j in range(0,64,4):
      words.append(x[j:j+4])
    AA, BB, CC , DD = a,b,c,d
    i = 0

    #Round 1
    s = [7, 12, 17, 22]
    
    for l in range(4):
      a = (b + left_rotate(a + F(b,c,d) + T[i] + int.from_bytes(words[i%16], byteorder='little'),s[0])) & 0xFFFFFFFF
      i += 1
      d = (a + left_rotate(d + F(a,b,c) + T[i] + int.from_bytes(words[i%16], byteorder='little'),s[1])) & 0xFFFFFFFF
      i += 1
      c = (d + left_rotate(c + F(d,a,b) + T[i] + int.from_bytes(words[i%16], byteorder='little'),s[2])) & 0xFFFFFFFF
      i += 1
      b = (c + left_rotate(b + F(c,d,a) + T[i] + int.from_bytes(words[i%16], byteorder='little'),s[3])) & 0xFFFFFFFF
      i += 1


    #Round 2
    s = [5, 9, 14, 20]
    f = lambda i: (5*i + 1)%16
    for l in range(4):
      a = (b + left_rotate(a + G(b,c,d) + T[i] + int.from_bytes(words[f(i%16)], byteorder='little'),s[0])) & 0xFFFFFFFF
      i += 1
      d = (a + left_rotate(d + G(a,b,c) + T[i] + int.from_bytes(words[f(i%16)], byteorder='little'),s[1])) & 0xFFFFFFFF
      i += 1
      c = (d + left_rotate(c + G(d,a,b) + T[i] + int.from_bytes(words[f(i&16)], byteorder='little'),s[2])) & 0xFFFFFFFF     
      i += 1
      b = (c + left_rotate(b + G(c,d,a) + T[i] + int.from_bytes(words[f(i%16)], byteorder='little'),s[3])) & 0xFFFFFFFF      
      i += 1


    #Round 3
    s = [4, 11, 16, 23]
    f = lambda i: (3*i + 5)%16
    for l in range(4):
      a = (b + left_rotate(a + H(b,c,d) + T[i] + int.from_bytes(words[f(i%16)], byteorder='little'),s[0])) & 0xFFFFFFFF
      i += 1
      d = (a + left_rotate(d + H(a,b,c) + T[i] + int.from_bytes(words[f(i%16)], byteorder='little'),s[1])) & 0xFFFFFFFF
      i += 1
      c = (d + left_rotate(c + H(d,a,b) + T[i] + int.from_bytes(words[f(i%16)], byteorder='little'),s[2])) & 0xFFFFFFFF
      i += 1
      b = (c + left_rotate(b + H(c,d,a) + T[i] + int.from_bytes(words[f(i%16)], byteorder='little'),s[3])) & 0xFFFFFFFF
      i += 1

    #Round 4
    s = [6, 10, 15, 21]
    f = lambda i: (7*i)%16
    for l in range(4):
      a = (b + left_rotate(a + I(b,c,d) + T[i] + int.from_bytes(words[f(i%16)], byteorder='little'),s[0])) & 0xFFFFFFFF
      i += 1
      d = (a + left_rotate(d + I(a,b,c) + T[i] + int.from_bytes(words[f(i%16)], byteorder='little'),s[1])) & 0xFFFFFFFF
      i += 1
      c = (d + left_rotate(c + I(d,a,b) + T[i] + int.from_bytes(words[f(i%16)], byteorder='little'),s[2])) & 0xFFFFFFFF
      i += 1
      b = (c + left_rotate(b + I(c,d,a) + T[i] + int.from_bytes(words[f(i%16)], byteorder='little'),s[3])) & 0xFFFFFFFF
      i += 1
    a += AA
    a &= 0xFFFFFFFF
    b += BB
    b &= 0xFFFFFFFF
    c += CC
    c &= 0xFFFFFFFF
    d += DD
    d &= 0xFFFFFFFF
  hash_pieces = [a,b,c,d]
  return md5_to_hex(sum(x<<(32*i) for i, x in enumerate(hash_pieces)))


mensaje = sys.argv[1]
h0 = int(sys.argv[2])
print(custom_md5(mensaje,h0),' <= "',mensaje,'"', sep='')

118ab80df76744795a0313d85b403e6e <= "jotramon@uc.cl"
